In [17]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [18]:
df=pd.read_csv('train.csv')

In [19]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [20]:
print((df.target==1).sum())
print((df.target==0).sum())

3271
4342


In [21]:
import re
import string
def remove_url(text):
    url=re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",text)
def remove_punct(text):
    translator=str.maketrans("","",string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
pa=re.compile(r"https?://(\S+|www)\.\S+")

for t in df.text:
    matches=pa.findall(t)
    for match in matches:
        print(t)
        print(matches)
        print(pa.sub(r"",t))
    if len(matches)>0:
        break



@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
['t']
@bbcmtd Wholesale Markets ablaze 


In [23]:
df['text']=df.text.apply(remove_url)

In [25]:
df['text']=df.text.apply(remove_punct)

In [26]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1
2,5,NaN,NaN,All residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1


In [27]:
for t in df.text:
    print(t)

Our Deeds are the Reason of this earthquake May ALLAH Forgive us all
Forest fire near La Ronge Sask Canada
All residents asked to shelter in place are being notified by officers No other evacuation or shelter in place orders are expected
13000 people receive wildfires evacuation orders in California 
Just got sent this photo from Ruby Alaska as smoke from wildfires pours into a school 
RockyFire Update  California Hwy 20 closed in both directions due to Lake County fire  CAfire wildfires
flood disaster Heavy rain causes flash flooding of streets in Manitou Colorado Springs areas
Im on top of the hill and I can see a fire in the woods
Theres an emergency evacuation happening now in the building across the street
Im afraid that the tornado is coming to our area
Three people died from the heat wave so far
Haha South Tampa is getting flooded hah WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK flooding
raining flooding Florida TampaBay Tampa 18 or 19 days Ive 

In [28]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raghu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [29]:
stop=set(stopwords.words('english'))
def remove_stopwords(text):
    filtered_words=[word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

In [30]:
df.text=df.text.apply(remove_stopwords)

In [31]:
df.text

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [32]:
from collections import Counter
def counter_word(text_col):
    count=Counter()
    for text in text_col.values:
        for word in text.split():
            count[word]+=1
    return count
counter=counter_word(df.text)

In [33]:
len(counter)

17971

In [34]:
counter.most_common(5)

[('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]

In [35]:
num_unique_words=len(counter)

In [36]:
train_size=int(df.shape[0]*0.8)
train_df=df[:train_size]
val_df=df[train_size:]
train_sentences=train_df.text.to_numpy()
train_labels=train_df.target.to_numpy()
val_sentences=val_df.text.to_numpy()
val_labels=val_df.target.to_numpy()

In [37]:
from keras.preprocessing.text import Tokenizer
token=Tokenizer(num_words=num_unique_words)
token.fit_on_texts(train_sentences)

In [38]:
word_index=token.word_index

In [39]:
word_index

{'like': 1,
 'amp': 2,
 'fire': 3,
 'im': 4,
 'get': 5,
 'via': 6,
 'new': 7,
 'people': 8,
 'news': 9,
 'dont': 10,
 'emergency': 11,
 'one': 12,
 '2': 13,
 'us': 14,
 'video': 15,
 'disaster': 16,
 'burning': 17,
 'body': 18,
 'would': 19,
 'buildings': 20,
 'police': 21,
 'crash': 22,
 'first': 23,
 'california': 24,
 'still': 25,
 'man': 26,
 'got': 27,
 'know': 28,
 'day': 29,
 'back': 30,
 'going': 31,
 'two': 32,
 'time': 33,
 'full': 34,
 'accident': 35,
 'see': 36,
 'world': 37,
 'attack': 38,
 'nuclear': 39,
 'youtube': 40,
 'may': 41,
 'love': 42,
 'go': 43,
 'rt': 44,
 'many': 45,
 'cant': 46,
 '3': 47,
 'watch': 48,
 'collapse': 49,
 'dead': 50,
 'today': 51,
 'car': 52,
 'mass': 53,
 'want': 54,
 'years': 55,
 'work': 56,
 'train': 57,
 'last': 58,
 'good': 59,
 'think': 60,
 'families': 61,
 'hiroshima': 62,
 'life': 63,
 'fires': 64,
 'best': 65,
 'could': 66,
 'say': 67,
 'u': 68,
 'death': 69,
 'hot': 70,
 'forest': 71,
 'way': 72,
 'killed': 73,
 'need': 74,
 'legion

In [76]:
train_sequences=token.texts_to_sequences(train_sentences)
val_sequences=token.texts_to_sequences(val_sentences)

In [77]:
print(train_sequences)

[[3739, 696, 235, 41, 1282, 3740, 14], [71, 3, 129, 576, 5670, 5671, 1283], [1448, 1186, 1882, 495, 5672, 1449, 116, 1882, 495, 976, 1187], [2243, 8, 3741, 1070, 116, 976, 24], [27, 1071, 358, 5673, 1635, 892, 1070, 5674, 91], [2244, 359, 24, 1188, 550, 609, 5675, 373, 1072, 374, 3, 5676, 1070], [99, 16, 610, 296, 977, 643, 117, 1636, 5677, 748, 1883, 1284], [4, 127, 2245, 36, 3, 5678], [236, 11, 116, 893, 432, 697, 577], [4, 2246, 2247, 180, 335], [520, 8, 395, 156, 297, 411], [749, 470, 2248, 138, 2249, 2813, 521, 611, 188, 470, 2248, 189, 189, 5679, 117], [2814, 117, 1884, 5680, 2248, 1285, 1450, 522, 256, 644, 2815], [99, 3742, 612, 1451, 3742], [111, 91, 336, 3743, 3744, 52, 22, 312], [433, 26], [42, 5681], [237, 1286], [52, 698], [5682], [2816], [894, 396], [42, 5683], [3745, 29], [5684], [5685, 46, 1885, 132], [1073, 58, 360], [42, 1886], [5686], [1, 3746], [257], [5687, 2250, 2817, 375], [157, 613, 750, 610, 895, 44], [5688, 312, 5689, 523, 139, 375, 802], [1287, 139, 375], [16

In [78]:
from keras.preprocessing.sequence import pad_sequences
max=20
train_padded=pad_sequences(train_sequences,max,padding='post',truncating='post')
val_padded=pad_sequences(val_sequences,max,padding='post',truncating='post')

In [79]:
print(train_padded)

[[ 3739   696   235 ...     0     0     0]
 [   71     3   129 ...     0     0     0]
 [ 1448  1186  1882 ...     0     0     0]
 ...
 [  151     1  1256 ...     0     0     0]
 [ 1256   448    15 ...     0     0     0]
 [15469   151   204 ...     0     0     0]]


In [80]:
reverse_word=dict([(idx,word) for word,idx in word_index.items() ])

In [81]:
reverse_word1={idx:word for word,idx in word_index.items()}

In [82]:
reverse_word1

{1: 'like',
 2: 'amp',
 3: 'fire',
 4: 'im',
 5: 'get',
 6: 'via',
 7: 'new',
 8: 'people',
 9: 'news',
 10: 'dont',
 11: 'emergency',
 12: 'one',
 13: '2',
 14: 'us',
 15: 'video',
 16: 'disaster',
 17: 'burning',
 18: 'body',
 19: 'would',
 20: 'buildings',
 21: 'police',
 22: 'crash',
 23: 'first',
 24: 'california',
 25: 'still',
 26: 'man',
 27: 'got',
 28: 'know',
 29: 'day',
 30: 'back',
 31: 'going',
 32: 'two',
 33: 'time',
 34: 'full',
 35: 'accident',
 36: 'see',
 37: 'world',
 38: 'attack',
 39: 'nuclear',
 40: 'youtube',
 41: 'may',
 42: 'love',
 43: 'go',
 44: 'rt',
 45: 'many',
 46: 'cant',
 47: '3',
 48: 'watch',
 49: 'collapse',
 50: 'dead',
 51: 'today',
 52: 'car',
 53: 'mass',
 54: 'want',
 55: 'years',
 56: 'work',
 57: 'train',
 58: 'last',
 59: 'good',
 60: 'think',
 61: 'families',
 62: 'hiroshima',
 63: 'life',
 64: 'fires',
 65: 'best',
 66: 'could',
 67: 'say',
 68: 'u',
 69: 'death',
 70: 'hot',
 71: 'forest',
 72: 'way',
 73: 'killed',
 74: 'need',
 75: 'le

In [83]:
def decode(sequence):
    return " ".join([reverse_word.get(idx,"?") for idx in sequence])


In [84]:
decode_text=decode(train_sequences[10])
print(train_sequences[10])
print(decode_text)

[520, 8, 395, 156, 297, 411]
three people died heat wave far


In [85]:
from keras import layers
model=keras.models.Sequential()
model.add(layers.Embedding(num_unique_words,32,input_length=max))

In [86]:
model.add(keras.layers.LSTM(64))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 32)            575072    
                                                                 
 lstm_7 (LSTM)               (None, 64)                24832     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 599969 (2.29 MB)
Trainable params: 599969 (2.29 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [89]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),optimizer=keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy'])

In [90]:
model.fit(train_padded,train_labels,epochs=5,batch_size=64,verbose=True)

Epoch 1/5


96/96 [==============================] - 3s 12ms/step - loss: 0.5252 - accuracy: 0.7388
Epoch 2/5
96/96 [==============================] - 1s 13ms/step - loss: 0.2693 - accuracy: 0.9056
Epoch 3/5
96/96 [==============================] - 1s 13ms/step - loss: 0.1560 - accuracy: 0.9486
Epoch 4/5
96/96 [==============================] - 1s 11ms/step - loss: 0.1137 - accuracy: 0.9612
Epoch 5/5
96/96 [==============================] - 1s 11ms/step - loss: 0.0963 - accuracy: 0.9693
